<h2 style='color:purple' align='center'>Naive Bayes Tutorial Part 1: Predicting survival from titanic crash</h2>

In [62]:
import pandas as pd

In [63]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [64]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [65]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [66]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [67]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [68]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


In [69]:
inputs.drop('Sex',axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0


In [70]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [71]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [72]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0
3,1,35.0,53.1000,1,0
4,3,35.0,8.0500,0,1


In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [74]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [75]:
model.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [76]:
model.score(X_test,y_test)

0.8059701492537313

In [77]:
X_test[0:10]

,Pclass,Age,Fare,female,male
676,3,24.500000,8.0500,0,1
457,1,29.699118,51.8625,1,0
284,1,29.699118,26.0000,0,1
766,1,29.699118,39.6000,0,1
866,2,27.000000,13.8583,1,0
88,1,23.000000,263.0000,1,0
66,2,29.000000,10.5000,1,0
330,3,29.699118,23.2500,1,0
583,1,36.000000,40.1250,0,1
662,1,47.000000,25.5875,0,1


In [78]:
y_test[0:10]

676    0
457    1
284    0
766    0
866    1
88     1
66     1
330    1
583    0
662    0
Name: Survived, dtype: int64

In [79]:
model.predict(X_test[0:10])

array([0, 1, 0, 0, 1, 1, 1, 1, 0, 0], dtype=int64)

In [80]:
model.predict_proba(X_test[:10])

array([[9.87222210e-01, 1.27777904e-02],
       [8.31796365e-03, 9.91682036e-01],
       [9.13476693e-01, 8.65233066e-02],
       [8.96861533e-01, 1.03138467e-01],
       [4.70160035e-02, 9.52983997e-01],
       [5.22796063e-13, 1.00000000e+00],
       [4.81892747e-02, 9.51810725e-01],
       [9.73780746e-02, 9.02621925e-01],
       [9.01835579e-01, 9.81644211e-02],
       [9.19345011e-01, 8.06549889e-02]])